In [39]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.examples.tutorials.mnist import input_data

In [40]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [41]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1
tf.reset_default_graph()

In [42]:
logs_path = "tmp/mnist/3"
if not os.path.exists(logs_path):
    os.makedirs(logs_path)

In [43]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [44]:
with tf.name_scope('Input'):
    x = tf.placeholder("float", [None, n_input],name="X-Input")
    y = tf.placeholder("float", [None, n_classes],name="Y-Input")

In [45]:
def build_three_layered_network():
    
    with tf.name_scope("Weights"):
        weights = {
            'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]),name="Hidden-Layer-1-Weights"),
            'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name="Hidden-Layer-2-Weights"),
            'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]),name="Output-Layer-Weights")
        }
    
    with tf.name_scope("Biases"):
        biases = {
            'b1': tf.Variable(tf.random_normal([n_hidden_1]),name="Hidden-Layer-1-Biases"),
            'b2': tf.Variable(tf.random_normal([n_hidden_2]),name="Hidden-Layer-2-Biases"),
            'out': tf.Variable(tf.random_normal([n_classes]),name="Output-Layer-Biases")
        }
    layer_1 = tf.add(tf.matmul(x, weights['h1'],name="Matmul-HL1"), biases['b1'],name="Bias-Add-HL1")
    layer_1 = tf.nn.relu(layer_1,name="Relu-HL1")
        # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2'],name="Matmul-HL2"), biases['b2'],name="Bias-Add-HL2")
    layer_2 = tf.nn.relu(layer_2,name="Relu-HL2")
        # Output layer with linear activation
    pred = tf.add(tf.matmul(layer_2, weights['out'],name="Matmul-Output"), biases['out'],name="Bias-Output")
    return pred


In [46]:
pred = build_three_layered_network()

In [47]:
# Define loss and optimizer

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y,name="Softmax-Cross-Entropt"),name="Reduce-Mean-Softmax-Cross-Entropt")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,name="Adam-Optimizer").minimize(cost)

In [48]:
init = tf.global_variables_initializer()

In [49]:
session = tf.Session()
session.run(init)

In [50]:
def optimize(training_epochs):
    writer = tf.summary.FileWriter(logs_path)
    writer.add_graph(tf.get_default_graph())
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary=session.run([optimizer,summary_op], feed_dict={x: batch_x,
                                                          y: batch_y})
            writer.add_summary(summary, epoch * total_batch + i)
        

In [51]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"),name="Accuracy")

In [52]:
tf.summary.scalar("cost", cost)
tf.summary.scalar("accuracy", accuracy)

summary_op = tf.summary.merge_all()

In [53]:
def print_accuracy():
    acc = session.run(accuracy,feed_dict={x: mnist.test.images,y: mnist.test.labels})
    print("Accuracy on test-set: {0:.1%}".format(acc))

In [54]:
optimize(1)
print_accuracy()

Accuracy on test-set: 85.3%


In [ ]:
#tensorboard --logdir=~/Documents/BlogDeepLearning/tmp/mnist/3 --port=6007